In [3]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [5]:
from trigger_utils.trigger_utils import get_kowalski_ztf_queue
import yaml

with open('../config/Credentials.yaml', 'r') as file:
    credentials = yaml.safe_load(file)
fritz_token = credentials['fritz_token']
ztf_allocation = credentials['allocation']

In [7]:
# print the current queue

current_ztf_queue = get_kowalski_ztf_queue(fritz_token, ztf_allocation)
current_ztf_queue

{'status': 'success',
 'data': {'queue_names': ['Caltech_Prince_2025-05-01',
   'Caltech_Prince_2025-05-03',
   'EP_2025-04-30_12',
   'EP_2025-04-30_14',
   'EP_2025-04-30_15',
   'EP_2025-04-30_16',
   'EP_2025-04-30_5',
   'EP_2025-04-30_6',
   'EP_2025-04-30_8',
   'EP_2025-04-30_9',
   'ToO_S250319bu_BBHBot_2025-05-01 01:25:32.352',
   'Twilight_2025-04-30_e',
   'Twilight_2025-04-30_m',
   'Twilight_2025-05-01_e',
   'Twilight_2025-05-01_m',
   'Twilight_2025-05-02_e',
   'Twilight_2025-05-02_m',
   'Twilight_2025-05-03_e',
   'Twilight_2025-05-03_m',
   'Twilight_2025-05-04_e',
   'Twilight_2025-05-04_m',
   'default',
   'fallback',
   'missed_obs']},
 'version': '1.4.0+fritz.86bd000'}

In [10]:
from trigger_utils.trigger_utils import get_plan_stats

# get a specific plan
gcnevent_id=13342
queuename="S250319bu_BBHBot_2025-04-03 01:18:21.260"
stats = get_plan_stats(gcnevent_id, queuename, fritz_token, mode='')

Already submitted to queue
Total time: 2940, probability: 0.7988238394848364
